**LAB 3. TOPIC MODELING**

TEXT NORMALIZATION

Below, we'll define a number of functions that perform various text "cleaning" jobs. After each function is defined, you can test that function by running it on a test sentence (object called test_text). At the end of the notebook, we combine all those function into one function called normolize_corpus.

We'll use the sentence below for testing the functions. It has punctuations signs, numbers, HTML markups, and other things to take care of:

In [1]:
test_text = "<p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>"

Get modules ready and available:

In [2]:
import sys

!{sys.executable} -m pip install html.parser
import html.parser
from html.parser import HTMLParser

import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

import unicodedata
import string

!{sys.executable} -m pip install pattern3
import pattern3


import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from pprint import pprint

# Plotting tools
!{sys.executable} -m pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

  Could not find a version that satisfies the requirement html.parser (from versions: )
No matching distribution found for html.parser
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 23.7MB 1.2MB/s ta 0:00:011   17% |█████▋                          | 4.2MB 12.4MB/s eta 0:00:02    69% |██████████████████████▎         | 16.5MB 10.0MB/s eta 0:00:01
    100% |████████████████████████████████| 419kB 9.3MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 5.8MB/s ta 0:00:011
    100% |████████████████████████████████| 194kB 9.2MB/s eta 0:00:01
    100% |████████████████████████████████| 4.1MB 3.6MB/s eta 0:00:01    70% |██████████████████████▋         | 2.9MB 6.9MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 4.7MB/s ta 0:00:01
    100% |████████████████████████████████| 5.6MB 3.0MB/s ta 0:00:011    56% |█████████████████

    100% |████████████████████████████████| 286kB 9.8MB/s eta 0:00:01
    100% |████████████████████████████████| 829kB 5.2MB/s ta 0:00:011
  Running setup.py bdist_wheel for pyLDAvis ... done
  Stored in directory: /Users/corrine/Library/Caches/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /Users/corrine/Library/Caches/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built pyLDAvis future
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Define functions for lemmatization and HTML parsing:

In [3]:
wnl = WordNetLemmatizer()
html_parser = HTMLParser()

Defining the list of word contractions:

In [4]:
contraction_mapping = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

Select the list of stopwords from NLTK and amend it by adding more stopwords to it:

In [5]:
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list = stopword_list + ['mr', 'mrs', 'come', 'go', 'get',
                                 'tell', 'listen', 'one', 'two', 'three',
                                 'four', 'five', 'six', 'seven', 'eight',
                                 'nine', 'zero', 'join', 'find', 'make',
                                 'say', 'ask', 'tell', 'see', 'try', 'back',
                                 'also']

Split text into word tokens:

In [6]:
def tokenize_text(text):
    tokens = nltk.word_tokenize(text) 
    tokens = [token.strip() for token in tokens]
    return tokens

print("Original:  ", test_text)
print("Processed: ", tokenize_text(test_text))

Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']


Expand contractions:

In [7]:
def expand_contractions(text, contraction_mapping):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text
 
print("Original:  ", test_text)
print("Processed: ", expand_contractions(test_text,contraction_mapping))


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>


Annotate text tokens with Part-Of-Speach tags:

In [8]:
def pos_tag_text(text_tokens):
    def penn_to_wn_tags(pos_tag):
        if pos_tag.startswith('J'):
            return wn.ADJ
        elif pos_tag.startswith('V'):
            return wn.VERB
        elif pos_tag.startswith('N'):
            return wn.NOUN
        elif pos_tag.startswith('R'):
            return wn.ADV
        else:
            return None  
    tagged_text = nltk.pos_tag(text_tokens)
    tagged_lower_text = [(word.lower(), penn_to_wn_tags(pos_tag))
                         for word, pos_tag in
                         tagged_text]
    return tagged_lower_text


print("Original:  ", test_text)
print("Processed: ", pos_tag_text(tokenize_text(test_text)))

Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), ('<', 'n'), ('/p', 'n'), ('>', 'n')]


Lemmatize text based on Part-Of-Speach (POS) tags: 

In [9]:
def lemmatize_text(text):
    pos_tagged_text = pos_tag_text(text)
    lemmatized_tokens = [wnl.lemmatize(word, pos_tag) if pos_tag
                         else word                     
                         for word, pos_tag in pos_tagged_text]
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text
 
print("Original:  ", test_text)
print("Processed: ", lemmatize_text(tokenize_text(test_text)))

Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  < p > the circus dog in a plissé skirt jump over python who be n't that large , just 3 foot long. < /p >


Remove special characters, such as punctuation marks:

In [10]:
def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub(' ', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text 

print("Original:  ", test_text)
print("Processed: ", remove_special_characters(test_text))

Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:    p   The circus dog in a plissé skirt jumped over Python who was n t that large   just 3 feet long     p  


Get rid of stopwords:

In [11]:
def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

print("Original:  ", test_text)
print("Processed: ", remove_stopwords(test_text))

Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  < p > The circus dog plissé skirt jumped Python n't large , 3 feet long. < /p >


Remove all non-text characters (numbers, etc.):

In [12]:
def keep_text_characters(text):
    filtered_tokens = []
    tokens = tokenize_text(text)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

print("Original:  ", test_text)
print("Processed: ", keep_text_characters(test_text))

Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  p The circus dog in a plissé skirt jumped over Python who was n't that large just feet long. /p


Clean up HTML markups: 

In [13]:
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ' '.join(self.fed)
    
def strip_html(text):
    html_stripper = MLStripper()
    html_stripper.feed(text)
    return html_stripper.get_data()

print("Original:  ", test_text)
print("Processed: ", strip_html(test_text))

Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.


Removing accents from characters:

In [14]:
def normalize_accented_characters(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8')
    return text

print("Original:  ", test_text)
print("Processed: ", normalize_accented_characters(test_text))

Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plisse skirt jumped over Python who wasn't that large, just 3 feet long.</p>


Putting all functions together:

In [15]:
def normalize_corpus(corpus, only_text_chars=True):
    normalized_corpus = []  
    for index, text in enumerate(corpus):
        text = normalize_accented_characters(text)
        text = html.unescape(text)
        text = strip_html(text)
        text = expand_contractions(text, contraction_mapping)
        text = tokenize_text(text)
        text = lemmatize_text(text)
        text = remove_special_characters(text)
        text = remove_stopwords(text)
        if only_text_chars:
            text = keep_text_characters(text)
        #text = tokenize_text(text)
        normalized_corpus.append(text)    
    return normalized_corpus

Create a small corpus consisting of 2 test sentences and testing the normalize_corpus function on it:

In [16]:
test_corpus = [test_text]
test_corpus.append(test_text)

print("Original:  ", test_corpus,"\n")
print("Processed: ", normalize_corpus(test_corpus))

Original:   ["<p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>", "<p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>"] 

Processed:  ['circus dog plisse skirt jump python large foot long', 'circus dog plisse skirt jump python large foot long']


** TOPIC MODELING: TOY CORPUS ** 
    
First, define a small function that would display the results of fitting a topic model:

In [17]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

We'll start by looking at the toy corpus on animals and programming we discussed last session:

In [18]:
toy_corpus = ["The fox jumps over the dog", 
              "The fox is very clever and quick", 
              "The dog is slow and lazy", 
              "The cat is smarter than the fox and the dog but it can never learn Java", 
              "Python is an excellent programming language", 
              "Java and Ruby are other programming languages", 
              "Python and Java are very popular programming languages", 
              "Python programs are smaller than Java programs"]

Normalize and create the "bag-of-words" representation of the corpus:

In [19]:
normalized_toy_corpus = normalize_corpus(toy_corpus)
bow_vectorizer = CountVectorizer()
bow_toy_corpus = bow_vectorizer.fit_transform(normalized_toy_corpus)
bow_feature_names_toy_corpus = bow_vectorizer.get_feature_names()

Have a look at the "bag-of-words" representation of your corpus (never hurts to know how you data look like!)

In [20]:
bow_table = pd.DataFrame(data = bow_toy_corpus.todense(), columns = bow_feature_names_toy_corpus)
bow_table.head

<bound method NDFrame.head of    cat  clever  dog  excellent  fox  java  jump  language  lazy  learn  never  \
0    0       0    1          0    1     0     1         0     0      0      0   
1    0       1    0          0    1     0     0         0     0      0      0   
2    0       0    1          0    0     0     0         0     1      0      0   
3    1       0    1          0    1     1     0         0     0      1      1   
4    0       0    0          1    0     0     0         1     0      0      0   
5    0       0    0          0    0     1     0         1     0      0      0   
6    0       0    0          0    0     1     0         1     0      0      0   
7    0       0    0          0    0     1     0         0     0      0      0   

   popular  program  programming  python  quick  ruby  slow  small  smarter  
0        0        0            0       0      0     0     0      0        0  
1        0        0            0       0      1     0     0      0        0  
2     

Select number of topics, sparsity parameters (alpha >0 and beta >0, set them to < 1 to induce sparsity) to identify and fit the topic model (LDA):

In [109]:
no_topics = 2
doc_topic_prior_toy = 0.1  # alpha > 0
topic_word_prior_toy = 0.01 # beta > 0
lda_toy_corpus = LatentDirichletAllocation(n_components=no_topics, 
                                           doc_topic_prior = doc_topic_prior_toy,
                                           topic_word_prior = topic_word_prior_toy).fit(bow_toy_corpus)

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Display results (we set that 10 most frequent words are displayed for each topic)

In [110]:
no_top_words = 10
display_topics(lda_toy_corpus, bow_feature_names_toy_corpus, no_top_words)

Topic 0:
dog fox lazy clever never cat slow quick java learn
Topic 1:
python language java programming program excellent small popular ruby fox


Look at topics: word vectors:

In [76]:
word_weights = lda_toy_corpus.components_ / lda_toy_corpus.components_.sum(axis=1)[:, np.newaxis]
topic_1_words = pd.DataFrame(data = word_weights[0], index = bow_feature_names_toy_corpus)
topic_2_words = pd.DataFrame(data = word_weights[1], index = bow_feature_names_toy_corpus)
# topic_3_words = pd.DataFrame(data = word_weights[2], index = bow_feature_names_toy_corpus)
tw = pd.concat([topic_1_words,topic_2_words],axis = 1)
tw.T

,cat,clever,dog,excellent,fox,java,jump,language,lazy,learn,never,popular,program,programming,python,quick,ruby,slow,small,smarter
0,0.075991,0.034971,0.077175,0.033885,0.080783,0.079472,0.032882,0.039219,0.031998,0.076874,0.076307,0.036654,0.033006,0.040757,0.035852,0.035777,0.034806,0.034996,0.032881,0.075715
0,0.031221,0.070207,0.113452,0.032231,0.108731,0.035175,0.068909,0.032133,0.072473,0.032892,0.033424,0.032364,0.034579,0.033991,0.034186,0.069500,0.031387,0.070670,0.030984,0.031489


**EXERCISE 1:**
1. Try adjusting the number of topics and see if you get more satisfied with the results.
No, it's more confusing.
2. With 2 topics, try adjusting alpha (doc_topic_prior) and beta (topic_word_prior) and observe the changes in topic representation.
3. Comment on your findings
It seems that the result is more satisfying when parameters are small.

** TOPIC MODELING: NEWS **

The dataset here is the one we used for doing classification. 
The newspaper blogposts have 4 topics: atheism, religion, computer graphics and space sciene. 
Of course, we will not use this information for topic modeling.

Download the data and set up the data:

In [24]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
dataset = fetch_20newsgroups(shuffle=True, 
                             random_state=1, 
                             categories = categories, 
                             remove=('headers', 'footers', 'quotes'))
news_corpus = dataset.data

Normalize the corpus and create "bag-of-words" representation of the data. We'll limit the number of features to 1000. 

NOTE: It will take a couple of minutes to get the data ready! 

In [25]:
no_features = 1000

normalized_corpus_news = normalize_corpus(news_corpus)

bow_vectorizer_news = CountVectorizer(max_features=no_features)
bow_news_corpus = bow_vectorizer_news.fit_transform(normalized_corpus_news)
bow_feature_names_news = bow_vectorizer_news.get_feature_names()

Set number of topics:

In [117]:
no_topics_news = 4

Run the topic model (LDA). NOTE: It will take a couple of minutes for the estimation to finish!

In [120]:
lda_news = LatentDirichletAllocation(n_components=no_topics_news, 
                                     max_iter=100,
                                    doc_topic_prior = 0.25,
                                    topic_word_prior = 0.01).fit(bow_news_corpus)

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Display results:

In [121]:
no_top_words_news = 10
display_topics(lda_news, bow_feature_names_news, no_top_words_news)

Topic 0:
image file use edu program software graphic format data jpeg
Topic 1:
would know people jesus think god like good thing time
Topic 2:
god people think would believe atheist argument religion use point
Topic 3:
space nasa launch satellite year system orbit use earth mission


**EXERCISE 2:**

1. Try 3 and 5 topics, do results get better or worse, in your opinion? 
3 is fine and 5 is worse.
2. Introduce sparsity parameters alpha and beta (doc_topic_prior and topic_word_prior) and see if can get better results.
Yes

** INTERACTIVE TOPIC VISUALIZATION **

Relevancy weight parameter - λ (0 ≤ λ ≤ 1): you can adjust it!

* small λ highlights potentially rare, but exclusive terms for the selected topic;
* large values of λ (near 1) highlight frequent, but not necessarily exclusive, terms for the selected topic;

    Relevancy = λ log[p(term | topic)] + (1 - λ) log[p(term | topic)/p(term)]

Additional information on how to use this visualization:
* http://www.kennyshirley.com/LDAvis/
* https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf


In [122]:
pyLDAvis.enable_notebook()
visualization_panel = pyLDAvis.sklearn.prepare(lda_news, bow_news_corpus, bow_vectorizer_news, mds='tsne')
visualization_panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1      -78.671196 -396.283112       1        1  28.387236
0      -33.778664   -7.443441       2        1  25.969431
3      138.194778 -224.309540       3        1  23.266617
2     -250.644653 -179.417053       4        1  22.376716, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
846   Default  1048.000000       space  1048.000000  30.0000  30.0000
415   Default   848.000000       image   848.000000  29.0000  29.0000
328   Default   545.000000        file   545.000000  28.0000  28.0000
575   Default   415.000000        nasa   415.000000  27.0000  27.0000
477   Default   399.000000      launch   399.000000  26.0000  26.0000
367   Default   789.000000         god   789.000000  25.0000  25.0000
268   Default   437.000000         edu   437.000000  24.0000  24.0000
449   Default   406.000000       jesus   406.000000  23.0000  23.0000
778   Default   285.000000   satellite   285.000000  22.0000  22.0000
833   Default   297.000000    software   297.000000  21.0000  21.0000
373   Default   294.000000     graphic   294.000000  20.0000  20.0000
700   Default   534.000000     program   534.000000  19.0000  19.0000
648   Default   798.000000      people   798.000000  18.0000  18.0000
74    Default   253.000000     atheist   253.000000  17.0000  17.0000
344   Default   280.000000      format   280.000000  16.0000  16.0000
623   Default   253.000000       orbit   253.000000  15.0000  15.0000
63    Default   245.000000    argument   245.000000  14.0000  14.0000
455   Default   260.000000        jpeg   260.000000  13.0000  13.0000
350   Default   256.000000         ftp   256.000000  12.0000  12.0000
554   Default   243.000000     mission   243.000000  11.0000  11.0000
156   Default   246.000000       color   246.000000  10.0000  10.0000
905   Default   758.000000       think   758.000000   9.0000   9.0000
207   Default   447.000000        data   447.000000   8.0000   8.0000
996   Default   442.000000        year   442.000000   7.0000   7.0000
95    Default   416.000000     believe   416.000000   6.0000   6.0000
73    Default   198.000000     atheism   198.000000   5.0000   5.0000
513   Default   242.000000        mail   242.000000   4.0000   4.0000
814   Default   201.000000     shuttle   201.000000   3.0000   3.0000
263   Default   281.000000       earth   281.000000   2.0000   2.0000
747   Default   301.000000    religion   301.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
747    Topic4   222.579750    religion   301.729376   1.1929  -4.7757
410    Topic4   152.302203       human   194.962210   1.2502  -5.1551
95     Topic4   278.976697     believe   416.794663   1.0957  -4.5498
707    Topic4    79.577483       prove    93.006718   1.3412  -5.8042
176    Topic4    90.274410  conclusion   109.204962   1.3068  -5.6781
367    Topic4   419.859180         god   789.190076   0.8661  -4.1410
655    Topic4   126.126410      person   169.502583   1.2016  -5.3437
573    Topic4   186.653611        must   292.804425   1.0469  -4.9517
298    Topic4   165.031260     example   265.831019   1.0204  -5.0748
923    Topic4   168.652017        true   282.515278   0.9813  -5.0531
950    Topic4   136.049517       value   206.174375   1.0814  -5.2679
648    Topic4   332.317917      people   798.213708   0.6209  -4.3749
905    Topic4   308.536925       think   758.905922   0.5971  -4.4491
149    Topic4   139.731282       claim   227.154066   1.0112  -5.2412
3      Topic4   100.694809      accept   133.885007   1.2123  -5.5689
673    Topic4   202.209007       point   494.450720   0.6030  -4.8716
127    Topic4   108.840312       cause   155.791754   1.1385  -5.4911
837    Topic4   153.028279   something   300.862112   0.8211  -5.1503
519    Topic4   188.933269        many 

**EXERCISE 3:**
    
Use the visualization and no_topics_news = 4 to answer the following question: Which topic is the most common topic in the corpus (toy can give a name to that topic)? List 5 most relevant and exclusive terms for that topic (with lambda = 0.2).

space is the most common one.
space, nasa, launch, satellite, orbit.